In [1]:
from IPython.display import display, Image, clear_output, HTML
import time
import random 
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import json

In [2]:
results_dict = {
    'filename': [],
    'correct': []
}

In [3]:
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [4]:
def display_test_img(img_file):
    style_str = f'width: 600px;'
    html_out = HTML(f"<img style='{style_str}' src={img_file}></img>")
    display(html_out)
    
display_test_img("myfigure.png") 

In [5]:
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [6]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return    

In [7]:
def run_ans_test(img_file, right_answer):
    blank_pic = Image("blank.png", width=600)

    btn1 = widgets.Button(description="blue")
    btn2 = widgets.Button(description="yellow")

    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    display_test_img(img_file)
    time.sleep(0.75)
    clear_output(wait=True)
    display(blank_pic)
    print("Input your answer: blue or yellow")
    display(btn1)
    display(btn2)

    result = wait_for_event(timeout=3)

    if result['description'] == right_answer:
        score = 1
    else:
        score = 0

    results_dict['filename'].append(img_file)
    
    if result['description'] == right_answer:
        results_dict['correct'].append('yes')
    else:
        results_dict['correct'].append('no')

    clear_output(wait=True)
    return score


In [8]:
def run_ans_full(num_repeats):
    files = ["12_vs_14.png", "12_vs_16.png", "15_vs_20.png", "16_vs_18.png", "18_vs_20.png", "18_vs_21.png", "9_vs_10.png", "9_vs_12.png", "10_vs_9_b.png", "12_vs_9_b.png", "14_vs_12_b.png", "16_vs_12_b.png", "18_vs_16_b.png", "20_vs_15_b.png", "20_vs_18_b.png", "21_vs_18_b.png"]
    answers = ["yellow", "yellow", "yellow", "yellow", "yellow", "yellow", "yellow", "yellow", "blue", "blue", "blue", "blue", "blue", "blue", "blue", "blue"]    
    
    total = 0
    score = []

    print("Welcome to the ANS test! For each trial, click the button which corresponds to the colour of the circles in the oval with the greatest number of circles, but be quick!") 

    print("To generate your unique ID, please input the first two letters of the first name of a childhood best friend, and the initial of your favourite actor. For example, Amy and Tom Cruise would give the ID AMTC")
    ans1 = input(">> ")
    print("What is your age in years? please give a number:")
    ans2 = input(">> ")
    print("What gender do you identify as? please input f for female, m for male, and nb for nonbinary: ")
    ans3 = input(">> ")

    for _ in range(num_repeats):
        indices = list(range(len(files)))
        random.shuffle(indices)
        
        for i in indices:
            score = run_ans_test(files[i], answers[i])
            total += score
        
    df = pd.DataFrame(results_dict)

    data_dict = {
        'id': ans1,
        'age': ans2,
        'gender': ans3,
        'score': total,
        'results': df.to_json()
    }

    form_url = "https://docs.google.com/forms/d/e/1FAIpQLScVS__ITc_Ju4ASqFvF-i14lLPBEdFCqLk-vU4RqIkBrK7VRw/viewform?usp=sf_link"
    send_to_google_form(data_dict, form_url)
           
    print("You scored", total)
    df = pd.read_csv("results.csv")
    print(df)
    print(data_dict)
    return data_dict

num_repeats = 4
data_dict = run_ans_full(num_repeats)


You scored 38
          filename correct
0   20_vs_18_b.png     yes
1   14_vs_12_b.png     yes
2      9_vs_10.png     yes
3    10_vs_9_b.png     yes
4      9_vs_12.png     yes
5   18_vs_16_b.png     yes
6     18_vs_21.png     yes
7     18_vs_20.png     yes
8     16_vs_18.png      no
9   16_vs_12_b.png     yes
10  20_vs_15_b.png     yes
11  21_vs_18_b.png     yes
12    12_vs_14.png     yes
13    15_vs_20.png      no
14   12_vs_9_b.png      no
15    12_vs_16.png     yes
{'id': 'SORZ', 'age': '21', 'gender': 'f', 'score': 38, 'results': '{"filename":{"0":"12_vs_14.png","1":"20_vs_15_b.png","2":"16_vs_18.png","3":"18_vs_20.png","4":"14_vs_12_b.png","5":"12_vs_16.png","6":"18_vs_21.png","7":"16_vs_12_b.png","8":"15_vs_20.png","9":"21_vs_18_b.png","10":"9_vs_12.png","11":"10_vs_9_b.png","12":"9_vs_10.png","13":"18_vs_16_b.png","14":"20_vs_18_b.png","15":"12_vs_9_b.png","16":"21_vs_18_b.png","17":"18_vs_16_b.png","18":"18_vs_21.png","19":"12_vs_9_b.png","20":"12_vs_16.png","21":"9_vs_12.png",

In [17]:
data_dict = {'id': 'maia', 'age': '21', 'gender': 'female', 'score': 10, 'results': '{"filename":{"0":"12_vs_16.png","1":"9_vs_10.png","2":"18_vs_20.png","3":"12_vs_9_b.png","4":"20_vs_15_b.png","5":"16_vs_12_b.png","6":"12_vs_14.png","7":"15_vs_20.png","8":"10_vs_9_b.png","9":"20_vs_18_b.png","10":"14_vs_12_b.png","11":"18_vs_16_b.png","12":"9_vs_12.png","13":"21_vs_18_b.png","14":"18_vs_21.png","15":"16_vs_18.png"},"correct":{"0":"yes","1":"no","2":"no","3":"yes","4":"no","5":"yes","6":"yes","7":"yes","8":"yes","9":"yes","10":"yes","11":"yes","12":"no","13":"yes","14":"no","15":"no"}}'}
form_url = "https://docs.google.com/forms/d/e/1FAIpQLScVS__ITc_Ju4ASqFvF-i14lLPBEdFCqLk-vU4RqIkBrK7VRw/viewform?usp=sf_link"

form_id = form_url[34:90]
view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

page = requests.get(view_form_url)
content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
content = content.text[27:-1]
result = json.loads(content)[1][1]
form_dict = {}

loaded_all = True
for item in result:
    if item[1] not in data_dict:
        print(f"Form item {item[1]} not found. Data not uploaded.")
        loaded_all = False
    form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]

post_result = requests.post(post_form_url, data=form_dict)





In [14]:
results_dict

{'filename': ['12_vs_16.png',
  '9_vs_10.png',
  '18_vs_20.png',
  '12_vs_9_b.png',
  '20_vs_15_b.png',
  '16_vs_12_b.png',
  '12_vs_14.png',
  '15_vs_20.png',
  '10_vs_9_b.png',
  '20_vs_18_b.png',
  '14_vs_12_b.png',
  '18_vs_16_b.png',
  '9_vs_12.png',
  '21_vs_18_b.png',
  '18_vs_21.png',
  '16_vs_18.png'],
 'correct': ['yes',
  'no',
  'no',
  'yes',
  'no',
  'yes',
  'yes',
  'yes',
  'yes',
  'yes',
  'yes',
  'yes',
  'no',
  'yes',
  'no',
  'no']}

In [10]:
df = pd.read_csv("results.csv")
print(df)
print(data_dict)

          filename correct
0   20_vs_18_b.png     yes
1   14_vs_12_b.png     yes
2      9_vs_10.png     yes
3    10_vs_9_b.png     yes
4      9_vs_12.png     yes
5   18_vs_16_b.png     yes
6     18_vs_21.png     yes
7     18_vs_20.png     yes
8     16_vs_18.png      no
9   16_vs_12_b.png     yes
10  20_vs_15_b.png     yes
11  21_vs_18_b.png     yes
12    12_vs_14.png     yes
13    15_vs_20.png      no
14   12_vs_9_b.png      no
15    12_vs_16.png     yes


NameError: name 'data_dict' is not defined

In [11]:
myjson = df.to_json()
myjson

'{"filename":{"0":"20_vs_18_b.png","1":"14_vs_12_b.png","2":"9_vs_10.png","3":"10_vs_9_b.png","4":"9_vs_12.png","5":"18_vs_16_b.png","6":"18_vs_21.png","7":"18_vs_20.png","8":"16_vs_18.png","9":"16_vs_12_b.png","10":"20_vs_15_b.png","11":"21_vs_18_b.png","12":"12_vs_14.png","13":"15_vs_20.png","14":"12_vs_9_b.png","15":"12_vs_16.png"},"correct":{"0":"yes","1":"yes","2":"yes","3":"yes","4":"yes","5":"yes","6":"yes","7":"yes","8":"no","9":"yes","10":"yes","11":"yes","12":"yes","13":"no","14":"no","15":"yes"}}'

In [11]:
newdf = pd.read_json(myjson)
newdf

/tmp/ipykernel_209610/2248632611.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  newdf = pd.read_json(myjson)


,filename,correct
0,20_vs_18_b.png,yes
1,14_vs_12_b.png,yes
2,9_vs_10.png,yes
3,10_vs_9_b.png,yes
4,9_vs_12.png,yes
5,18_vs_16_b.png,yes
6,18_vs_21.png,yes
7,18_vs_20.png,yes
8,16_vs_18.png,no
9,16_vs_12_b.png,yes


In [12]:
data_dict

NameError: name 'data_dict' is not defined